Komaljeet Kaur,
Victoriia Biragova,
Monsurat Adejumo,
Harvir Kaur


Problem 9. What are the expected proportions of substances sent to landfills compared to substances sent for treatment or recycling in the next 5 years.

Importing the dataset with read_csv and displaying the data

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

In [78]:
df = pd.read_csv('Filled_columns.csv')

In [79]:
df.head()

,Reporting_Year / Année,NPRI_ID / No_INRP,Number of employees,Company_Name,Facility_Name / Installation,NAICS / Code_SCIAN,NAICS Title / Titre Code_SCIAN,PROVINCE,City,Latitude,...,Off-site Transfers for Recycling - Recovery of Organic Substances (not solvents),Off-site Transfers for Recycling - Recovery of Pollution Abatement Residues,Off-site Transfers for Recycling - Recovery of Solvents,Off-site Transfers for Recycling - Refining or Re-use of Used Oil,Off-site Transfers for Treatment Prior to Final Disposal - Biological Treatment,Off-site Transfers for Treatment Prior to Final Disposal - Chemical Treatment,Off-site Transfers for Treatment Prior to Final Disposal - Incineration / Thermal,Off-site Transfers for Treatment Prior to Final Disposal - Municipal Sewage Treatment Plant,Off-site Transfers for Treatment Prior to Final Disposal - Physical Treatment,On-site Disposal - Landfill
0,2000,4,500.0,pine falls paper company,pine falls,322122,newsprint mills,mb,pine falls,50.5686,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,2.6400
1,2000,4,500.0,pine falls paper company,pine falls,322122,newsprint mills,mb,pine falls,50.5686,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0057
2,2000,7,45.0,"borden chimie canada, inc.",borden chimie - laval,325210,resin and synthetic rubber manufacturing,qc,laval,45.5797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0000
3,2000,7,45.0,"borden chimie canada, inc.",borden chimie - laval,325210,resin and synthetic rubber manufacturing,qc,laval,45.5797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.011,0.0,0.0,0.0000
4,2000,7,45.0,"borden chimie canada, inc.",borden chimie - laval,325210,resin and synthetic rubber manufacturing,qc,laval,45.5797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.011,0.0,0.0,0.0000


In [80]:
df.dtypes

,0
Reporting_Year / Année,int64
NPRI_ID / No_INRP,int64
Number of employees,float64
Company_Name,object
Facility_Name / Installation,object
NAICS / Code_SCIAN,int64
NAICS Title / Titre Code_SCIAN,object
PROVINCE,object
City,object
Latitude,float64


check if the Reporting_Year / Année is in a day time format

Converting column 'Reporting_Year / Année' into datetime format if it's not converted

In [61]:
df['Reporting_Year / Année'] = pd.to_datetime(df['Reporting_Year / Année'], format='%Y', errors='coerce').dt.year

Organizes time-series waste data by year and pollutant type, handles missing values, aggregates data for analysis, and classifies waste disposal amounts into None, Low, Medium, or High based on quantiles.

In [62]:

# Selecting relevant columns for time series analysis
time_series_df = df[[
    "Reporting_Year / Année",  # Time dimension
    "NPRI_ID / No_INRP",       # Unique facility identifier
    "Substance Name (English) / Nom de substance (Anglais)",  # Pollutant type
    "Off-site Disposal - Landfill",
    "On-site Disposal - Landfill",
    "Off-site Transfers for Recycling - Energy Recovery",
    "Off-site Transfers for Recycling - Other",
    "Off-site Transfers for Recycling - Recovery of Inorganic Materials (not metals)",
    "Off-site Transfers for Recycling - Recovery of Acids and Bases",
    "Off-site Transfers for Recycling - Recovery of Catalysts",
    "Off-site Transfers for Recycling - Recovery of Metals and Metal Compounds",
    "Off-site Transfers for Recycling - Recovery of Organic Substances (not solvents)",
    "Off-site Transfers for Recycling - Recovery of Pollution Abatement Residues",
    "Off-site Transfers for Recycling - Recovery of Solvents",
    "Off-site Transfers for Recycling - Refining or Re-use of Used Oil",
    "Off-site Transfers for Treatment Prior to Final Disposal - Incineration / Thermal",
    "Off-site Transfers for Treatment Prior to Final Disposal - Chemical Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Biological Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Physical Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Municipal Sewage Treatment Plant"
]]



# Fill missing values with 0
time_series_df.fillna(0, inplace=True)

# Aggregate by year and substance
time_series_agg = time_series_df.groupby(["Reporting_Year / Année", "Substance Name (English) / Nom de substance (Anglais)"]).sum().reset_index()

# Classify each waste type separately
categories = [
    "Off-site Disposal - Landfill",
    "On-site Disposal - Landfill",
    "Off-site Transfers for Recycling - Energy Recovery",
    "Off-site Transfers for Recycling - Other",
    "Off-site Transfers for Recycling - Recovery of Inorganic Materials (not metals)",
    "Off-site Transfers for Recycling - Recovery of Acids and Bases",
    "Off-site Transfers for Recycling - Recovery of Catalysts",
    "Off-site Transfers for Recycling - Recovery of Metals and Metal Compounds",
    "Off-site Transfers for Recycling - Recovery of Organic Substances (not solvents)",
    "Off-site Transfers for Recycling - Recovery of Pollution Abatement Residues",
    "Off-site Transfers for Recycling - Recovery of Solvents",
    "Off-site Transfers for Recycling - Refining or Re-use of Used Oil",
    "Off-site Transfers for Treatment Prior to Final Disposal - Incineration / Thermal",
    "Off-site Transfers for Treatment Prior to Final Disposal - Chemical Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Biological Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Physical Treatment",
    "Off-site Transfers for Treatment Prior to Final Disposal - Municipal Sewage Treatment Plant"
]

def classify_value(series, value):
    if pd.isna(value) or value == 0:
        return "None"
    elif value <= series.quantile(0.33):
        return "Low"
    elif value <= series.quantile(0.66):
        return "Medium"
    else:
        return "High"

for category in categories:
    time_series_agg[f"{category}_Class"] = time_series_agg[category].apply(lambda x: classify_value(time_series_agg[category], x))

# Save the processed dataset
#time_series_agg.to_csv("processed_time_series_data.csv", index=False)


<ipython-input-62-3078ffe4f085>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_series_df.fillna(0, inplace=True)


 Convert categorical waste classifications ("None", "Low", "Medium", "High") into numerical values (0, 1, 2, 3) for easier processing

In [63]:
category_mapping = {"None": 0, "Low": 1, "Medium": 2, "High": 3}

for category in categories:
    time_series_agg[f"{category}_Class"] = time_series_agg[f"{category}_Class"].map(category_mapping)


This code defines features (X) and the target variable (y) for a machine learning model.

In [64]:
# Define features by dropping non-relevant columns ( year and substance name)
X = time_series_agg.drop(columns=["Reporting_Year / Année", "Substance Name (English) / Nom de substance (Anglais)"])
y = time_series_agg["Off-site Disposal - Landfill_Class"]  # Example target


This code trains a Random Forest Classifier to predict "Off-site Disposal - Landfill_Class" based on the selected features.

In [81]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00        50
           2       1.00      1.00      1.00       240
           3       1.00      1.00      1.00       247

    accuracy                           1.00       727
   macro avg       1.00      1.00      1.00       727
weighted avg       1.00      1.00      1.00       727



This code performs cross-validation to evaluate the stability and generalization of the Random Forest Classifier.

In [83]:
# Perform 5-fold cross-validation to evaluate model performance
cv_scores = cross_val_score(clf, X, y, cv=5)
print("Cross-Validation Scores:", cv_scores) # Print accuracy scores from each fold
print("Mean Accuracy:", cv_scores.mean()) # Compute and display the average accuracy across all folds


Cross-Validation Scores: [0.97661623 0.97936726 0.94085282 0.9656121  0.95867769]
Mean Accuracy: 0.9642252208214444


In [84]:

overlap = pd.merge(X_train, X_test, how="inner")
print(f"Number of overlapping rows between train and test: {len(overlap)}")


Number of overlapping rows between train and test: 49


This code checks if there are duplicate rows between the training (X_train) and testing (X_test) datasets after splitting the data. This ensures the train-test split is effective and there is no data leakage.

In [85]:
# Find overlapping rows
overlapping_indices = X_test.index.intersection(X_train.index)

# Drop overlapping rows from X_test
X_test = X_test.drop(index=overlapping_indices)
y_test = y_test.drop(index=overlapping_indices)

# Check again for overlap
overlap_final = X_train.index.intersection(X_test.index)
print(f" Final number of overlapping rows: {len(overlap_final)}")  # Should be 0


 Final number of overlapping rows: 0


This code re-trains the Random Forest classifier after ensuring the train-test split is clean (i.e., no duplicate samples between X_train and X_test). It then evaluates the model’s performance on the corrected test set.

In [86]:
# Train the classifier again
clf = RandomForestClassifier(n_estimators=100, max_depth=5)
clf.fit(X_train, y_train)

# Make predictions
y_pred_fixed = clf.predict(X_test)

# Evaluate model performance
print("New Accuracy (after fixing leakage):", accuracy_score(y_test, y_pred_fixed))
print("\nClassification Report:\n", classification_report(y_test, y_pred_fixed))


New Accuracy (after fixing leakage): 0.9601100412654745

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       190
           1       1.00      0.44      0.61        50
           2       0.90      1.00      0.94       240
           3       1.00      1.00      1.00       247

    accuracy                           0.96       727
   macro avg       0.97      0.86      0.89       727
weighted avg       0.96      0.96      0.95       727

